# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/Benchmarks/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data_set = 'Criteo'  #@param {type: "string"}

#cleaned data without non-attack values
X = pd.read_csv(root_path+data_set+'.csv', usecols=[i for i in range(2, 40)])
Y = pd.read_csv(root_path+data_set+'.csv', usecols=[1])

In [0]:
from sklearn.preprocessing import StandardScaler
X=X.astype('float')
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

# Feature Engineering

In [5]:
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import fowlkes_mallows_score as scorer
score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y[Y.columns[0]].unique())-1, random_state=0, max_iter=10)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = scorer(Y[Y.columns[0]].values,pred)
  score.append(s)
  

# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 



    
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]
  
  

# imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(n_estimators=10,random_state=0, n_jobs=-1, max_depth=5)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 
  

  
# best score calcuation
index = accuracy.index(max(accuracy))

X = X2.iloc[:,0:len(X.columns)-index]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed whe

# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

In [7]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [8]:

x = list(zip(X.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X.columns)):
    print(x[i])
    if (i==len(X.columns)-1):
      print("features",i+1)

(5, 0.06909760757311174)
(4, 0.052125933312606614)
(8, 0.04666827302318185)
(15, 0.046090606958554395)
(28, 0.04564713031904659)
(16, 0.045303440267957074)
(24, 0.04506782743210533)
(6, 0.04390057274764325)
(22, 0.04214729077856077)
(2, 0.042068985063572595)
(19, 0.04094320123365416)
(1, 0.03992335320410816)
(7, 0.03900704550925537)
(12, 0.038907923395260634)
(25, 0.03871716908012846)
(23, 0.03805403865496645)
(27, 0.03789479347759988)
(3, 0.032882082679760415)
(14, 0.03236000659936077)
(10, 0.032284292814874375)
(0, 0.031865289443340154)
(13, 0.02187597857346023)
(11, 0.020223140721939977)
(20, 0.019266714298294674)
(26, 0.018455797174952163)
(17, 0.01586972959227072)
(18, 0.014220090936496923)
(9, 0.009019838632285362)
(21, 0.00011184650165098926)
features 29


In [9]:
print(clf.score(x_train, y_train), "train")

0.980082380134921 train


In [10]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

              precision    recall  f1-score   support

           0     0.7362    0.9041    0.8116    103827
           1     0.5817    0.2918    0.3886     47484

    accuracy                         0.7119    151311
   macro avg     0.6590    0.5979    0.6001    151311
weighted avg     0.6877    0.7119    0.6788    151311

